## Part 1


In [2]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import math
import torch 
import torch.utils.data as data
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.modules.activation

In [3]:
link = 'https://raw.githubusercontent.com/propublica/compas-analysis/master/compas-scores-two-years.csv'
readlink = pd.read_csv(link)

In [5]:
#GPU Computation
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cpu


# Logistic Regression

"Participants saw a short description of a defendant that included the defendant’s sex, age, and previous criminal history, but not their race"

I will use the sex and age of the defendant to train the regression model. For sex, I will turn all data into integer values first to make them easily iterable

In [ ]:
#get sex from data
x_axis = readlink[["sex"]].copy()

#convert sex to int type
arr = []
for i in x_axis["sex"]:
  if i == "Female":
    arr.append(0)
  else:
    arr.append(1)

#append to all data
x_axis["sex"] = arr

In [ ]:
#add all remaining integer values
x_axis["age"] = readlink[["age"]].copy()

In [ ]:
#set y axis as the two year recidivism
y_axis = readlink[["two_year_recid"]].copy()

In [ ]:
#move data to the device 
new_x_axis = torch.tensor(x_axis.values).to(device)
new_y_axis = torch.tensor(y_axis.values).to(device)

"The results in columns (A), (B), and (C) correspond to the average testing accuracy over 1000 random 80%/20% training/testing splits."

In [ ]:
#find the index at which to split the data
val = np.floor(len(new_x_axis)*0.8)

#first 80% of the data
x_train = new_x_axis[:val.astype(int)].to(device)
y_train = new_y_axis[:val.astype(int)].to(device)

#last 20% of the data
x_test = new_x_axis[val.astype(int):].to(device)
y_test = new_y_axis[val.astype(int):].to(device)

In [ ]:
#training and testing data initialization: used from pytorch github tutorial
class TrainandTestDataset(data.Dataset):
    def __init__(self, x, y):
        super(TrainandTestDataset, self).__init__()
        assert x.shape[0] == y.shape[0]
        self.x = x
        self.y = y

    def __len__(self):
        return self.y.shape[0]

    def __getitem__(self, index):
        return self.x[index], self.y[index]

In [ ]:
trainloader = torch.utils.data.DataLoader(TrainandTestDataset(x_train, y_train), batch_size=32, shuffle = True)
testloader = torch.utils.data.DataLoader(TrainandTestDataset(x_test, y_test), batch_size=32, shuffle = False)

In [ ]:
#Logistic Regression Class
class LogisticRegression(nn.Module):
  def __init__(self, n_input, n_output):
    super(LogisticRegression, self).__init__()
    self.linear = nn.Linear(n_input, n_output)
  def forward(self,x):
    result = torch.sigmoid(self.linear(x))
    return result

In [ ]:
# declare the model
model = LogisticRegression(2, 1).to(device = 'cuda')

# define the criterion
criterion = nn.MSELoss()

# select the optimizer and pass to it the parameters of the model it will optimize
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)

In [ ]:
epochs = 1000
# training loop
for epoch in range(epochs):
    for i, (x_i, y_i) in enumerate(trainloader):
        optimizer.zero_grad()           # cleans the gradients   
        y_hat_i = model(x_i.float())            # forward pass
        loss = criterion(y_hat_i, y_i.float())  # compute the loss and perform the backward pass
        loss.backward()                 # computes the gradients
        optimizer.step()                # update the parameters

    if epoch % 20 == 0:
      print("epoch:", epoch, "loss=", loss.item())

epoch: 0 loss= 0.21140483021736145
epoch: 20 loss= 0.19770236313343048
epoch: 40 loss= 0.2765422761440277
epoch: 60 loss= 0.2996476888656616
epoch: 80 loss= 0.25924918055534363
epoch: 100 loss= 0.24272462725639343
epoch: 120 loss= 0.25439873337745667
epoch: 140 loss= 0.29320311546325684
epoch: 160 loss= 0.22930343449115753
epoch: 180 loss= 0.24006009101867676
epoch: 200 loss= 0.22071579098701477
epoch: 220 loss= 0.1891176998615265
epoch: 240 loss= 0.20026583969593048
epoch: 260 loss= 0.32082152366638184
epoch: 280 loss= 0.20555579662322998
epoch: 300 loss= 0.30765458941459656
epoch: 320 loss= 0.2087249457836151
epoch: 340 loss= 0.2704949975013733
epoch: 360 loss= 0.2545343339443207
epoch: 380 loss= 0.20040848851203918
epoch: 400 loss= 0.19837163388729095
epoch: 420 loss= 0.26728036999702454
epoch: 440 loss= 0.34621644020080566
epoch: 460 loss= 0.27221614122390747
epoch: 480 loss= 0.22132067382335663
epoch: 500 loss= 0.3074251711368561
epoch: 520 loss= 0.21839582920074463
epoch: 540 los

In [ ]:
# testing
with torch.no_grad():
    model.eval()
    total_loss = 0.
    for k, (x_k, y_k) in enumerate(testloader):
        y_hat_k = model(x_k.float())
        loss_test = criterion(y_hat_k, y_k.float())
        total_loss += float(loss_test)

print(total_loss)

10.987534761428833


# FPR Parity

To calculate the FPR parity, we need to import race data from the provided url

In [ ]:
y2_pred = model(x_test.float())

In [ ]:
#get race from data
race = readlink[["race"]].copy()

#convert race to int type
arr = []
for i in race["race"]:
  if i == "African-American":
    arr.append(0)
  elif i == "Caucasian":
    arr.append(1)
  else:
    arr.append(-1)

#append to all data
race["race"] = arr

In [ ]:
#move data to the device 
new_race = torch.tensor(race.values).to(device)

In [ ]:
#Find the total recidivated rates of Caucasian and African-Americans
b = 0
w = 0
br = 0
wr = 0

for i in range(len(new_race)):
  if new_race[i] == 0:
    b += 1
  if new_race[i] == 1:
    w += 1
  if new_race[i] == 0 and new_y_axis[i] == 1:
    br += 1
  if new_race[i] == 1 and new_y_axis[i] == 1:
    wr += 1

print("total number of defendants: ", len(new_race))
print("number of black defendants in the database: ", b)
print("number of white defendants in the database: ", w)
print("recidivated black defendants: ", br/b*100)
print("recidivated white defendants: ", wr/w*100)

total number of defendants:  7214
number of black defendants in the database:  3696
number of white defendants in the database:  2454
recidivated black defendants:  51.433982683982684
recidivated white defendants:  39.36430317848411


Note: As calculated above, the dataset can be concluded to be partially skewed, considering the significantly larger percentage of black defendants that are present in the data. As well, the percentage recidivated - grouped by race - is significantly different, with a percent difference of about 11%. These calculations can also be used to compare the models predictions to actual data present in the dataset.

In [ ]:
#split data based on paper: 80% training and 20% test

#find the index at which to split the data
val = np.floor(len(new_race)*0.8)

#first 80% of the data
race_train = new_race[:val.astype(int)].to(device)

#last 20% of the data
race_test = new_race[val.astype(int):].to(device)

In this calculation, let 50% be used as the threshold (chosen arbitrarily)

In [ ]:
#Find # of African Americans that are predicted to reoffend vs. who actually reoffend
actual = 0 #True Positive
were_predicted = 0 #False Positive
not_predicted = 0 #True Negative
wrong_predict = 0 #False Negative

for i in range(len(x_test)):
  if race_test[i] == 0: #If African-American
    if y2_pred[i] >= 0.5: #If predicted to reoffend
      if y_test[i] == 1: #did reoffend
        actual +=1
      if y_test[i] == 0: #did not reoffend
        were_predicted += 1
    if y2_pred[i] < 0.5: 
      if y_test[i] == 0: #did not reoffend 
        not_predicted += 1
      if y_test[i] == 1: #did reoffend
        wrong_predict += 1

print("# of African-Americans that reoffend and were predicted to reoffend: ", actual)
print("# of African-Americans that did not reoffend and were predicted to not reoffend: ", not_predicted)
print("# of African-Americans that did not reoffend but were predicted to reoffend: ", were_predicted)
print("# of African-Americans that did reoffend but were predicted not to reoffend: ", wrong_predict)
print("PPV: ", actual/(actual + were_predicted)*100, "%")
print("NPV: ", not_predicted/(not_predicted + wrong_predict)*100, "%")
print("False Positive Parity: ", were_predicted/(were_predicted + not_predicted)*100, "%")

# of African-Americans that reoffend and were predicted to reoffend:  233
# of African-Americans that did not reoffend and were predicted to not reoffend:  184
# of African-Americans that did not reoffend but were predicted to reoffend:  165
# of African-Americans that did reoffend but were predicted not to reoffend:  162
PPV:  58.5427135678392 %
NPV:  53.179190751445084 %
False Positive Parity:  47.277936962750715 %


In [ ]:
#Find # of Caucasian Americans that are predicted to reoffend vs. who actually reoffend
actual = 0 #True Positive
were_predicted = 0 #False Positive
not_predicted = 0 #True Negative
wrong_predict = 0 #False Negative

for i in range(len(x_test)):
  if race_test[i] == 1: #If Caucasian
    if y2_pred[i] >= 0.5: #If predicted to reoffend
      if y_test[i] == 1: #did reoffend
        actual +=1
      if y_test[i] == 0: #did not reoffend
        were_predicted += 1
    if y2_pred[i] < 0.5: 
      if y_test[i] == 0: #did not reoffend 
        not_predicted += 1
      if y_test[i] == 1: #did reoffend
        wrong_predict += 1

print("# of Caucasians that reoffend and were predicted to reoffend: ", actual)
print("# of Caucasians that did not reoffend and were predicted to not reoffend: ", not_predicted)
print("# of Caucasians that did not reoffend but were predicted to reoffend: ", were_predicted)
print("# of Caucasians that did reoffend but were predicted not to reoffend: ", wrong_predict)
print("PPV: ", actual/(actual + were_predicted)*100, "%")
print("NPV: ", not_predicted/(not_predicted + wrong_predict)*100, "%")
print("False Positive Parity: ", were_predicted/(were_predicted + not_predicted)*100, "%")

# of Caucasians that reoffend and were predicted to reoffend:  86
# of Caucasians that did not reoffend and were predicted to not reoffend:  212
# of Caucasians that did not reoffend but were predicted to reoffend:  86
# of Caucasians that did reoffend but were predicted not to reoffend:  92
PPV:  50.0 %
NPV:  69.73684210526315 %
False Positive Parity:  28.859060402684566 %


The claim that was discussed in the article stated, "the likelihood of recidivism for any given score is the same regardless of race (calibration)."

As seen above in the calculations carried out to determine the calibration and recidivism of both black and white defendants, it is clear that FPR parity has a difference that does not satisfy the bounds.

Following the paper of Corbett-Davies and Goel, adjusting the threshold can lead to satisfied FPR parity (but dissatisfied calibration). Next, we will attempt to adjust the threshold value to imitate this

In [ ]:
#adjusting thresholds: use 60%

#Find # of African Americans that are predicted to reoffend vs. who actually reoffend
actual = 0 #True Positive
were_predicted = 0 #False Positive
not_predicted = 0 #True Negative
wrong_predict = 0 #False Negative

for i in range(len(x_test)):
  if race_test[i] == 0: #If African-American
    if y2_pred[i] >= 0.6: #If predicted to reoffend
      if y_test[i] == 1: #did reoffend
        actual +=1
      elif y_test[i] == 0: #did not reoffend
        were_predicted += 1
    elif y2_pred[i] < 0.6: 
      if y_test[i] == 0: #did not reoffend 
        not_predicted += 1
      if y_test[i] == 1: #did reoffend
        wrong_predict += 1

print("# of African-Americans that reoffend: ", actual)
print("# of African-Americans that did not reoffend: ", not_predicted)
print("# of African-Americans that did not reoffend but were predicted to reoffend: ", were_predicted)
print("# of African-Americans that did reoffend but were predicted not to reoffend: ", wrong_predict)
print("PPV: ", actual/(actual + were_predicted)*100, "%")
print("NPV: ", not_predicted/(not_predicted + wrong_predict)*100, "%")
print("False Positive Parity: ", were_predicted/(were_predicted + not_predicted)*100, "%")

# of African-Americans that reoffend:  60
# of African-Americans that did not reoffend:  313
# of African-Americans that did not reoffend but were predicted to reoffend:  36
# of African-Americans that did reoffend but were predicted not to reoffend:  335
PPV:  62.5 %
NPV:  48.30246913580247 %
False Positive Parity:  10.315186246418339 %


In [ ]:
#adjusting thresholds: use 60%

#Find # of Caucasian Americans that are predicted to reoffend vs. who actually reoffend
actual = 0 #True Positive
were_predicted = 0 #False Positive
not_predicted = 0 #True Negative
wrong_predict = 0 #False Negative

for i in range(len(x_test)):
  if race_test[i] == 1: #If Caucasian
    if y2_pred[i] >= 0.6: #If predicted to reoffend
      if y_test[i] == 1: #did reoffend
        actual +=1
      elif y_test[i] == 0: #did not reoffend
        were_predicted += 1
    elif y2_pred[i] < 0.6: 
      if y_test[i] == 0: #did not reoffend 
        not_predicted += 1
      if y_test[i] == 1: #did reoffend
        wrong_predict += 1

print("# of Caucasians that reoffend: ", actual)
print("# of Caucasians that did not reoffend: ", not_predicted)
print("# of Caucasians that did not reoffend but were predicted to reoffend: ", were_predicted)
print("# of Caucasians that did reoffend but were predicted not to reoffend: ", wrong_predict)
print("PPV: ", actual/(actual + were_predicted)*100, "%")
print("NPV: ", not_predicted/(not_predicted + wrong_predict)*100, "%")
print("False Positive Parity: ", were_predicted/(were_predicted + not_predicted)*100, "%")

# of Caucasians that reoffend:  20
# of Caucasians that did not reoffend:  282
# of Caucasians that did not reoffend but were predicted to reoffend:  16
# of Caucasians that did reoffend but were predicted not to reoffend:  158
PPV:  55.55555555555556 %
NPV:  64.0909090909091 %
False Positive Parity:  5.369127516778524 %


As seen above, adjusting the threshold merely 10% leads to a larger calibration difference and better FPR parity difference.

# Part 2

Since there are over 400 charge descriptions, use one hot encoding to classify them

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [ ]:
#Use one hot encodings for charge descriptions
arr = []
charge_description = readlink[["c_charge_desc"]].copy()

for i in charge_description["c_charge_desc"]:
  arr.append(i)

arr = np.array(arr)

#Label Encoder will turn these values into integer values
int_vals = LabelEncoder().fit_transform(arr)

#One Hot Encoder will create the respective one hot vectors
one_hot_vector = OneHotEncoder(sparse = False).fit_transform(int_vals.reshape(7214, 1))

In [ ]:
x2_axis = readlink[["sex"]].copy()

#convert sex to int type
arr = []
for i in x2_axis["sex"]:
  if i == "Female":
    arr.append(0)
  else:
    arr.append(1)

In [ ]:
x2_axis = readlink[["c_charge_degree"]].copy()

#convert degree to int type
arr2 = []
for i in x2_axis["c_charge_degree"]:
  if i == "F":
    arr2.append(0)
  else:
    arr2.append(1)

In [ ]:
#append to all data
x2_axis["sex"] = arr
x2_axis["c_charge_degree"] = arr2

In [ ]:
#get all other integer valued data
x2_axis[["age", "juv_fel_count","juv_misd_count", "priors_count"]] = readlink[["age", "juv_fel_count","juv_misd_count", "priors_count"]].copy()
y2_axis = readlink[["two_year_recid"]].copy()

In [ ]:
#get race from data
race = readlink[["race"]].copy()

#convert race to int type
arr3 = []
for i in race["race"]:
  if i == "African-American":
    arr3.append(0)
  elif i == "Caucasian":
    arr3.append(1)
  else:
    arr3.append(-1)

#append to all data
race["race"] = arr3

In [ ]:
x2_axis = x2_axis.to_numpy()

In [ ]:
empty_arr = np.empty((7214, 444))

for i in range(len(x2_axis)):
  empty_arr[i] = np.concatenate((x2_axis[i], one_hot_vector[i]))

In [ ]:
#move data to the device 
new_x_axis = torch.from_numpy(empty_arr).to(device)
new_y_axis = torch.tensor(y2_axis.values).to(device)

In [ ]:
#split data based on paper: 80% training and 20% test

#find the indices at which to split the data
val = np.floor(len(new_x_axis)*0.8)

#first 80% of the data
x_train = new_x_axis[:val.astype(int)].to(device)
y_train = new_y_axis[:val.astype(int)].to(device)

#last 20% of the data
x_test = new_x_axis[val.astype(int):].to(device)
y_test = new_y_axis[val.astype(int):].to(device)

In [ ]:
race_data = torch.tensor(race.values).to(device)

#find the indices at which to split the data
val = np.floor(len(race_data)*0.8)

racetrain_dataset = race_data[:val.astype(int)].to(device)
racetest_dataset = race_data[val.astype(int):].to(device)

In [ ]:
class MainNetwork(nn.Module):
  def __init__(self):
    super(MainNetwork, self).__init__()
    self.network = nn.Sequential(nn.Linear(444, 1), nn.Sigmoid(), nn.Linear(1, 1))
  def forward(self, x):
    result = self.network(x)
    return result

class Adversarial(nn.Module):
  def __init__(self):
    super(Adversarial, self).__init__()
    self.network = nn.Sequential(nn.Linear(1, 1), nn.Sigmoid())
  def forward(self, x):
    result = self.network(x)
    return result

Note that the adversarial learning procedure would need to be iterative: the adversary is optimized, and then the whole network N is optimized, in a loop.

In [ ]:
#adversary
train_loader2 = torch.utils.data.DataLoader(TrainandTestDataset(x_train, racetrain_dataset), batch_size=32, shuffle = True)
test_loader2 = torch.utils.data.DataLoader(TrainandTestDataset(x_test, racetest_dataset), batch_size=32, shuffle = False)

In [ ]:
# declare the model
adversary_model = Adversarial().to(device)

# define the criterion
adversary_criterion = nn.MSELoss()

# select the optimizer and pass to it the parameters of the model it will optimize
adversary_optimizer = torch.optim.Adam(adversary_model.parameters(), lr = 0.01)

In [ ]:
#network
train_loader = torch.utils.data.DataLoader(TrainandTestDataset(x_train, y_train), batch_size=32, shuffle = True)
test_loader = torch.utils.data.DataLoader(TrainandTestDataset(x_test, y_test), batch_size=32, shuffle = False)

In [ ]:
# declare the model
network_model = MainNetwork().to(device)

# define the criterion
network_criterion = nn.MSELoss()

# select the optimizer and pass to it the parameters of the model it will optimize
network_optimizer = torch.optim.Adam(list(network_model.parameters()) + list(adversary_model.parameters()), lr = 0.01)

In [ ]:
#Adversary
epochs = 1000

# training loop
for epoch in range(epochs):
        adversary_optimizer.zero_grad()           # cleans the gradients    

        a = torch.sigmoid(network_model(x_train.float()))            # forward pass
        b = adversary_model(a)            # forward pass
  
        adversary_loss = adversary_criterion(b, racetrain_dataset.float())  # compute the loss and perform the backward pass
        adversary_loss.backward()                 # computes the gradients
  
        adversary_optimizer.step()                # update the parameters

        if epoch % 20 == 0:
          print("epoch:", epoch, "loss=", adversary_loss.item())

epoch: 0 loss= 0.5745607018470764
epoch: 20 loss= 0.5249937772750854
epoch: 40 loss= 0.49114662408828735
epoch: 60 loss= 0.47149041295051575
epoch: 80 loss= 0.46106547117233276
epoch: 100 loss= 0.45568031072616577
epoch: 120 loss= 0.452875018119812
epoch: 140 loss= 0.4513852894306183
epoch: 160 loss= 0.4505802392959595
epoch: 180 loss= 0.45014020800590515
epoch: 200 loss= 0.4498986303806305
epoch: 220 loss= 0.44976624846458435
epoch: 240 loss= 0.4496941566467285
epoch: 260 loss= 0.44965535402297974
epoch: 280 loss= 0.44963476061820984
epoch: 300 loss= 0.44962403178215027
epoch: 320 loss= 0.44961848855018616
epoch: 340 loss= 0.4496156573295593
epoch: 360 loss= 0.44961410760879517
epoch: 380 loss= 0.4496132731437683
epoch: 400 loss= 0.44961273670196533
epoch: 420 loss= 0.4496123492717743
epoch: 440 loss= 0.44961199164390564
epoch: 460 loss= 0.44961175322532654
epoch: 480 loss= 0.4496114253997803
epoch: 500 loss= 0.4496111273765564
epoch: 520 loss= 0.4496108293533325
epoch: 540 loss= 0.44

In [ ]:
#Main Network 
epochs = 1000

# training loop
for epoch in range(epochs):
  network_optimizer.zero_grad()

  a = torch.sigmoid(network_model(x_train.float())) 
  b = adversary_model(a)

  network_loss = network_criterion(a, y_train.float())
  network_loss.backward(retain_graph = True)
  
  network_optimizer.step()

  if (epoch) % 20 == 0: 
    print("epoch:", epoch, "\tNetwork loss=", network_loss.item())


epoch: 0 	Network loss= 0.27019622921943665
epoch: 20 	Network loss= 0.24481698870658875
epoch: 40 	Network loss= 0.23268704116344452
epoch: 60 	Network loss= 0.22402913868427277
epoch: 80 	Network loss= 0.21756288409233093
epoch: 100 	Network loss= 0.2123938351869583
epoch: 120 	Network loss= 0.2081780880689621
epoch: 140 	Network loss= 0.20477011799812317
epoch: 160 	Network loss= 0.2020874321460724
epoch: 180 	Network loss= 0.20000535249710083
epoch: 200 	Network loss= 0.19839654862880707
epoch: 220 	Network loss= 0.1971520334482193
epoch: 240 	Network loss= 0.19618217647075653
epoch: 260 	Network loss= 0.19542206823825836
epoch: 280 	Network loss= 0.19482585787773132
epoch: 300 	Network loss= 0.19435547292232513
epoch: 320 	Network loss= 0.19397689402103424
epoch: 340 	Network loss= 0.19366015493869781
epoch: 360 	Network loss= 0.19337698817253113
epoch: 380 	Network loss= 0.19311535358428955
epoch: 400 	Network loss= 0.19288693368434906
epoch: 420 	Network loss= 0.1926925182342529

In [ ]:
#Find # of African-Americans that are predicted to reoffend vs. who actually reoffend
actual = 0 #True Positive
were_predicted = 0 #False Positive
not_predicted = 0 #True Negative
wrong_predict = 0 #False Negative

for i in range(len(x_test)):
  if race_test[i] == 0: #If African-American
    if a[i] >= 0.5: #If predicted to reoffend
      if y_test[i] == 1: #did reoffend
        actual +=1
      elif y_test[i] == 0: #did not reoffend 
        were_predicted += 1
    elif a[i] < 0.5: 
      if y_test[i] == 0: #did not reoffend 
        not_predicted += 1
      if y_test[i] == 1: #did reoffend
        wrong_predict += 1

print("# of African-American that reoffend: ", actual)
print("# of African-American that did not reoffend: ", not_predicted)
print("# of African-American that did not reoffend but were predicted to reoffend: ", were_predicted)
print("# of African-American that did reoffend but were predicted not to reoffend: ", wrong_predict)
print("PPV: ", actual/(actual + were_predicted)*100, "%")
print("NPV: ", not_predicted/(not_predicted + wrong_predict)*100, "%")
print("False Positive Parity: ", were_predicted/(were_predicted + not_predicted)*100, "%")

# of African-American that reoffend:  138
# of African-American that did not reoffend:  198
# of African-American that did not reoffend but were predicted to reoffend:  151
# of African-American that did reoffend but were predicted not to reoffend:  257
PPV:  47.75086505190311 %
NPV:  43.51648351648352 %
False Positive Parity:  43.26647564469914 %


In [ ]:
#Find # of Caucasian Americans that are predicted to reoffend vs. who actually reoffend
actual = 0 #True Positive
were_predicted = 0 #False Positive
not_predicted = 0 #True Negative
wrong_predict = 0 #False Negative

for i in range(len(x_test)):
  if race_test[i] == 1: #If Caucasian
    if a[i] >= 0.5: #If predicted to reoffend
      if y_test[i] == 1: #did reoffend
        actual +=1
      elif y_test[i] == 0: #did not reoffend
        were_predicted += 1
    elif a[i] < 0.5: 
      if y_test[i] == 0: #did not reoffend 
        not_predicted += 1
      if y_test[i] == 1: #did reoffend
        wrong_predict += 1

print("# of Caucasians that reoffend: ", actual)
print("# of Caucasians that did not reoffend: ", not_predicted)
print("# of Caucasians that did not reoffend but were predicted to reoffend: ", were_predicted)
print("# of Caucasians that did reoffend but were predicted not to reoffend: ", wrong_predict)
print("PPV: ", actual/(actual + were_predicted)*100, "%")
print("NPV: ", not_predicted/(not_predicted + wrong_predict)*100, "%")
print("False Positive Parity: ", were_predicted/(were_predicted + not_predicted)*100, "%")

# of Caucasians that reoffend:  66
# of Caucasians that did not reoffend:  181
# of Caucasians that did not reoffend but were predicted to reoffend:  117
# of Caucasians that did reoffend but were predicted not to reoffend:  112
PPV:  36.0655737704918 %
NPV:  61.774744027303754 %
False Positive Parity:  39.261744966442954 %


The following seven features were used: 
- age
- juv_fel_count
- juv_misd_count
- priors_count
- sex
- c_charge_degree
- c_charge_desc

For the first five listed above, nothing was done. For sex and c_charge_desc, these string values were converted to integer values, which was easy due to their binary manner. The last one was difficult as there were over 400 unique charges. In order to combat this issue, one hot vectors were used where each unique charge was given its own unique vector that could be used in training and testing.

The adversarial model followed the given procedure:
- Train the adversaral learning model: Linear->Sigmoid
- Train the network: Linear->Sigmoid->Linear->Adversal->Sigmoid

A threshold of 50% was used for this trial, in order to compare the results to part 1, where only two attributes were used. As seen above, the FPR parity is < 5%, although the process in which the parity was calculated is the same as done in part 1. This shows that this model as well as increasing the number of features can improve these results by providing consistency between races in prediction calculations.